# Creacion de SQL script

Este codigo tiene por objetivo tomar los datos del json original, procesarlos y pasarlos a un script SQL para generar el esquema de tablas
1. obtencion de los JSON
2. definicion de esquema de BBDD
3. filtrado de pacientes, solo tomaremos los paciences contenidos en la tabla evolucion
4. modificacion de datos de pacientes
5. almacenar los datos de evoluciones y pacientes en archivos

## 1. Obtencion de los JSON

In [6]:
import os
import pandas as pd
from dotenv import load_dotenv
import json

load_dotenv()

# obtener pacientes
pacientes_json = []
with open("./json/pacientes.json", "r", encoding="utf-8") as f:
    pacientes_json = json.load(f)
pacientes = pacientes_json[2]["data"]
    
# obtener evoluciones
evoluciones_json = []
with open("./json/evolucion.json", "r", encoding="utf-8") as f:
    evoluciones_json = json.load(f)
evoluciones = evoluciones_json[2]["data"]


## 2. Definicion de esquema de BBDD

CREATE TABLE evolucion (
    id INTEGER PRIMARY KEY,
    id_paciente INTEGER,
    fecha DATE,
    edad INTEGER,
    uni_edad VARCHAR(20),
    edad_anios FLOAT,
    edad_texto VARCHAR(50),
    peso FLOAT,
    talla FLOAT,
    imc FLOAT,
    pc FLOAT,
    motivo VARCHAR(255),
    conducta TEXT,
    FOREIGN KEY (id_paciente) REFERENCES paciente(id_paciente)
);

CREATE TABLE paciente (
    id_paciente INTEGER PRIMARY KEY,
    apellido VARCHAR(100),
    nombre VARCHAR(100),
    fecha_nac DATE,
    sexo VARCHAR(10),
    edad VARCHAR(20),
    dni VARCHAR(20),
    localidad VARCHAR(100),
    obra_social VARCHAR(100),
    afiliado_nro VARCHAR(50),
    telefono VARCHAR(100),
    telefono_numero VARCHAR(20),
    email VARCHAR(100),
    especialidad VARCHAR(100),
    diagnostico VARCHAR(255),
    enfermedad_base VARCHAR(255),
    ant_perinatales TEXT,
    ant_familiares TEXT,
    registro VARCHAR(50),
    fecha_reg VARCHAR(50)
);

## 3. filtrado de pacientes

Para hacer esto debenmos tomar solo los pacientes que tienen consultas

In [7]:
# iterar sobre las consultas
ids_pacientes_con_evoluciones = set()
for evolucion in evoluciones:
    id_paciente = evolucion["id_paciente"]
    if id_paciente not in ids_pacientes_con_evoluciones:
        ids_pacientes_con_evoluciones.add(id_paciente)
# filtrar pacientes
pacientes_con_evoluciones = [p for p in pacientes if p["id_paciente"] in ids_pacientes_con_evoluciones] 

# crear dataframes
df_pacientes = pd.DataFrame(pacientes_con_evoluciones)
df_evoluciones = pd.DataFrame(evoluciones)

print(f"Pacientes original cantidad: {pacientes.__len__()}")
print(f"Pacientes con evoluciones cantidad: {pacientes_con_evoluciones.__len__()}")
print(f"Evoluciones cantidad: {evoluciones.__len__()}")
print("----------------------------------------")




Pacientes original cantidad: 315
Pacientes con evoluciones cantidad: 70
Evoluciones cantidad: 79
----------------------------------------


# 4. Modificar los datos personales reales
Modificar los datos pidiendo al LLM que genere datos ficticios
- apellido
- nombre
- dni
- afiliado_nro
- telefono
- telefono_numero
- email

In [23]:
# instalar libreria de lanchain y openai
%pip install langchain langchain-openai langchain_community pydantic pprint


ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
import os

## 1. instalar librerias
## 2. importar librerias
## 3. iterar sobre pacientes
## 4. llamar a la api para modificar los datos personales

# funcion para crear paciente ficticio
def createPacienteFicticio(paciente, datos_ficticios):
    paciente["apellido"] = datos_ficticios.apellido
    paciente["nombre"] = datos_ficticios.nombre
    paciente["dni"] = datos_ficticios.dni
    paciente["afiliado_nro"] = datos_ficticios.afiliado_nro
    paciente["telefono"] = datos_ficticios.telefono
    paciente["telefono_numero"] = datos_ficticios.telefono_numero
    paciente["email"] = datos_ficticios.email
    return paciente


# Configurar la API key de OpenAI
openai_api_key = os.getenv("OPENAI_API_KEY")
llm_model = "gpt-4o-mini"
llm = ChatOpenAI(openai_api_key=openai_api_key, model = llm_model, temperature=0.7)

# configurar el parser de salida
class PacienteFicticio(BaseModel):
    apellido: str = Field(...)
    nombre: str = Field(...)
    dni: str = Field(...)
    afiliado_nro: str = Field(...)
    telefono: str = Field(...)
    telefono_numero: str = Field(...)
    email: str = Field(...)
    
# estructurando la salida del modelo
llm_with_structure = llm.with_structured_output(PacienteFicticio)

# Definir el prompt para anonimizar datos personales
prompt = PromptTemplate(
    input_variables=["apellido", "nombre", "dni", "afiliado_nro", "telefono", "telefono_numero", "email"],
    template=(
        """
            Dado los siguientes datos personales de un paciente:
            Apellido: {apellido}
            Nombre: {nombre}
            DNI: {dni}
            Afiliado Nro: {afiliado_nro}
            Teléfono: {telefono}
            Teléfono Número: {telefono_numero}
            Email: {email}
            Genera datos ficticios para cada campo, manteniendo el formato y tipo de dato.

            <example>
                Apellido: Perez -> Apellido ficticio: Gomez // si el apellido es compuesto, usa un apellido compuesto ficticio. Usa apellidos de nacionalidad similar al original
                Nombre: Juan -> Nombre ficticio: Mateo // si el nombre es compuesto, usa un nombre compuesto ficticio. Usa nombres respetando el genero
                DNI: 12.345.678 -> DNI ficticio: 12.678.345 // intercambia la posicion de las ultimas 3 cifras por 3 cifras anteriores
                Afiliado Nro: 123456789 -> Afiliado Nro ficticio: 123456987 // invierte el orden de los ultimos 3 digitos. Si hay mas de un numero de afiliado identifica cada uno y aplica la misma logica
                Teléfono: 1234-567890 -> Teléfono ficticio: 4321-098765 // inventate cualquier cosa. si hay mas de un numero de telefono, identifica cada uno y aplica la misma logica
                Teléfono Número: 1234567890 -> Teléfono Número ficticio: 0987654321 // inventate cualquier cosa. si hay mas de un numero de telefono, identifica cada uno y aplica la misma logica
                Email: juan.perez@email.com -> Email ficticio: mateo.gomez@email.com // usa el nombre y apellido ficticio separados por un punto y el dominio @mail.com
            </example>
            
            Aclaraciones: si un campo viene vacio, dejalo vacio en la respuesta. Si un campo no tiene el formato esperado, inventate un valor con el formato correcto.
        """
    )
)

# Iterar sobre los pacientes y anonimizar datos personales
pacientes_ficticios = []
for index, paciente in enumerate(pacientes_con_evoluciones):
    input_data = {
        "apellido": paciente["apellido"],
        "nombre": paciente["nombre"],
        "dni": paciente["dni"],
        "afiliado_nro": paciente["afiliado_nro"],
        "telefono": paciente["telefono"],
        "telefono_numero": paciente["telefono_numero"],
        "email": paciente["email"],
    }
    print(index, end=" - ")
    if index % 20 == 0:
        print()
    chain = prompt | llm_with_structure
    respuesta = chain.invoke(input = input_data)
    pacientes_ficticios.append(createPacienteFicticio(paciente, respuesta))





0 - 
1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 
21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - 33 - 34 - 35 - 36 - 37 - 38 - 39 - 40 - 
41 - 42 - 43 - 44 - 45 - 46 - 47 - 48 - 49 - 50 - 51 - 52 - 53 - 54 - 55 - 56 - 57 - 58 - 59 - 60 - 
61 - 62 - 63 - 64 - 65 - 66 - 67 - 68 - 69 - 

# 5. Guardar los pacientes ficticios en CSV

In [40]:
df_pacientes_ficticios = pd.DataFrame(pacientes_ficticios)
df_pacientes_ficticios.to_csv("./pacientes_ficticios.csv", index=False)

## 6. Guardar las evolucuiones en CSV

In [9]:
## almacenar EVOLUCIONES en archivos CSV

# print("Guardando los pacientes filtrados en ./data/pacientes.csv")
# # Guardar pacientes en CSV, preservando saltos de línea
# df_pacientes.to_csv("./pacientes.csv", index=False)

# Nota: pandas convierte automáticamente '\r\n' en saltos de línea reales en el CSV.
# Esto puede romper la estructura del CSV si abres el archivo con un editor de texto o Excel,
# ya que los saltos de línea dentro de los campos pueden hacer que una fila se divida en varias.
# Para evitar esto, puedes reemplazar los saltos de línea por espacios o algún otro símbolo antes de guardar:

# df_pacientes.replace({r'\r\n': ' | ', r'\n': ' | ', r'\r': ' | '}, regex=True).to_csv("./pacientes.csv", index=False)

print("Guardando las evoluciones en ./data/evoluciones.csv")
df_evoluciones.to_csv("./evoluciones.csv", index=False)


Guardando los pacientes filtrados en ./data/pacientes.csv
Guardando las evoluciones en ./data/evoluciones.csv
